[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/jun-ogura/preprocessing-pet-images/blob/master/animal_PET_img_other_functions.ipynb)

# Pythonを使ったPETデータ前処理
## - Other functions -
---
　ここでは、animal_PET_img_processing.ipynb で触れられていない機能を記載します。

主に以下の関数が含まれます。<br>
<br>
- ExtractSlices : <br>
　　シングルフレーム画像の任意のスライスのみを抽出するための関数<br>
<br>
- CreateMeanImgToMultiFrm : <br>
　　マルチフレームデータ（4D NIfTI）から任意のフレームを抽出し、<br>
　　その平均画像を作製する関数<br>
<br>
- AffineTransformByAlignedImg : <br>
　　3D Slicerで位置合わせが終わった画像のヘッダー情報をもとに<br>
　　回転・平行移動を他の画像データに適用するための関数<br>
<br>
- create_SUVimg2PrecentID_per_ml_img : <br>
　　SUV画像を%ID/ml画像にする変換する関数<br>
<br>
- createSUVrImg_PMOD / createSUVrImg_SAMIT : <br>
　　SUVr（リファレンスVOIで標準化した）画像を作製するための関数<br>
<br>
- createMeanImg : <br>
　　群の平均画像を作製するための関数<br>
<br>
- InvertAxis : <br>
　　反転をするための関数(アフィン行列を修正して見た目だけ反転させる方法<br>
　　→　元画像は反転しない)<br>
<br>
- InvertAxisV2 : <br>
　　反転をするための関数(アフィン行列は編集せずに元画像を反転させる方法)<br>
<br>
- CalcDifferenceImg : <br>
　　差分画像を作製するための関数<br>
<br>
- CalcRatioImg : <br>
　　割合（百分率）画像を作製するための関数<br>
<br>
- CalcMaskImg : <br>
　　マスク適用画像を作製するための関数<br>
<br>

滅多に使用しないものもありますが、参考までに載せています。

### Reference Link

- [ 7.0.0. Reference documentation: all nilearn functions ](https://nilearn.github.io/modules/reference.html#reference-documentation-all-nilearn-functions)

- [ 7.5.8. nilearn.image.index_img ](https://nilearn.github.io/modules/generated/nilearn.image.index_img.html#nilearn.image.index_img)

- [ 7.5.13. nilearn.image.mean_img ](https://nilearn.github.io/modules/generated/nilearn.image.mean_img.html#nilearn.image.mean_img)

# 01. パッケージの読み込み
---
各関数で必要となるパッケージは以下です。
- nilearn
- nibabel
- numpy

In [ ]:
import os
from nilearn.image import load_img, mean_img, index_img
import numpy as np
import nibabel as nib

from nibabel.processing import smooth_image
from nilearn.image import load_img, mean_img, index_img, resample_img, concat_imgs, iter_img
from nilearn.regions import img_to_signals_labels
import nipype.interfaces.spm as spm
import pandas as pd

ModuleNotFoundError: ignored

# 02. シングルフレーム画像の任意のスライスのみを抽出するための関数
---

- ClairvivoPETで2匹同時撮像を行ったとき、再構成後の画像から1匹のみの画像を取り出すためには該当個体のみが映っているスライスだけを抽出すればよい。

- 画像解析を行う上で、余計なものが映りこんでいると精度が落ちる恐れがあり、被検体以外の領域が無駄に多いとそれだけ処理に時間がかかります。

- ここでは<u>**2匹が向い合せに配置されており、体軸方向に対してスライスが行われている**</u>ことを前提に話を進めます（もちろんそれ以外でも使用できます）。

- また、ClairvivoPETはスライス枚数が213であり、中心スライスは107となっています。

In [ ]:
def ExtractSlices(inputDir, inputFile, startS, stopS, outputDir, prefix, suffix):
    #inputDir : 入力ファイルのあるディレクトリ
    #inputFile : 入力ファイル（4D NIfTI形式）
    #startS : 抽出する画像の開始スライス（一番手前が0）
    #stopS : 抽出する画像の終了スライス
    #outputDir : 出力ディレクトリの指定
    #prefix : 出力ファイル名の接頭語
    #suffix : 出力ファイル名の接尾語
    
    # import 4D NIfTI
    niimg = load_img(inputDir + '/' + inputFile + '.nii')
    #print(niimg.shape)

    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    ImgArryData = niimg.get_fdata()

    # スライスの抽出
    if len(ImgArryData.shape)==3:
        ExtractedArrayNii = ImgArryData[:,:,startS:stopS]   # 3D
    elif len(ImgArryData.shape)==4:
        ExtractedArrayNii = ImgArryData[:,:,startS:stopS,:] # 4D
    else:
        pass
    print(ExtractedArrayNii.shape)
    
    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換
    ConvertImg = nib.Nifti1Image(ExtractedArrayNii, None, header=new_header)
    
    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + inputFile + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

以下に具体的な使用例を示します（animal_PET_img_processing.ipynbと同様）。

<br>
このJupyter Notebookファイルと同じフォルダ階層に以下のフォルダが存在し、<br>
そのフォルダ内にNIfTIファイルが保存されているとします。

    ./static/test_data.nii

最初に関数に渡す引数（inputDir、outputDir、prefix、suffix）を指定します。<br>
関数を実行すると処理結果の画像ファイルが引数で指定されたフォルダに出力されます。<br>
<br>
各引数の説明は上記の関数内に記載されていますのでそちらをご確認ください（以降の関数も同様）。

In [ ]:
inputDir = './static'
outputDir = './static'
prefix = 'p_'
suffix = ''

# 手前の個体
ExtractSlices(inputDir, 'test_data', 0, 107, outputDir, prefix, suffix)

# 奥側の個体
ExtractSlices(inputDir, 'test_data', 90, 197, outputDir, prefix, suffix)

# 03. マルチフレームデータ（4D NIfTI）から任意のフレームを抽出し、その平均画像を作製する関数
---


In [ ]:
def createMeanFramesImg(inputDir, inputFile, frameRnage, outputDir, prefix, suffix):
    #inputDir : 入力ファイルのあるディレクトリ
    #inputFile : 入力ファイル（4D NIfTI形式）
    #frameRnage : どこからどこまでのフレームを使って加算平均するかを指定（slice()関数で指定する）
    #outputDir : 出力ディレクトリの指定
    #prefix : 出力ファイル名の接頭語
    #suffix : 出力ファイル名の接尾語
    
    # import 4D NIfTI
    niimg = load_img(inputDir + '/' + inputFile + '.nii')
    tmpArrayNiimg = niimg.get_fdata()

    # get single image
    if len(tmpArrayNiimg.shape)==3:
        print('Can not use the 3D image. Use a 4D image only.')
        
    elif len(tmpArrayNiimg.shape)==4:
        multi_frame_img = index_img(niimg, frameRnage) # 4D data (maluti frames)
        single_mean_image = mean_img(multi_frame_img)
        
    else:
        pass
    
    single_mean_imageArray = single_mean_image.get_fdata()

    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換
    ConvertImg = nib.Nifti1Image(single_mean_imageArray, None, header=new_header)
    
    # infomation
    tmpArray = multi_frame_img.get_fdata()
    print('Complete: /' + inputFile + '.nii  ---   ' + 'shape: ' + str(tmpArray.shape))
    
    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + inputFile + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

このJupyter Notebookファイルと同じフォルダ階層に以下のフォルダが存在し、<br>
そのフォルダ内にマルチフレームのNIfTIファイルが保存されているとします。

    ./dynamic/multiframe_test_data.nii

最初に関数に渡す引数（inputDir、inputFile、frameRnage、outputDir、prefix、suffix）を指定します。

以下の例ではフレーム8～30を抽出し、その平均画像を作製する場合となります。

引数`frameRnage`は`slice()`を使って指定しますが、第一引数は指定したい<u>フレーム番号から1引いた数</u>になるため注意してください（Pythonのインデックスが0スタートのため）。

In [ ]:
inputDir = './dynamic'
outputDir = './dynamic'
prefix = 'm_'
suffix = ''

createMeanFramesImg(inputDir, 'multiframe_test_data', slice(7, 30), outputDir, prefix, suffix)

# 04. 位置合わせの終わった画像のヘッダー情報をもとに回転・平行移動を他の画像データに適用するための関数
---

- AffineTransform()（[Pythonを使ったPETデータ前処理- Main functions -の02. 回転・平行移動をするための関数](https://github.com/jun-ogura/preprocessing-pet-images/blob/main/animal_PET_img_processing_ver01.ipynb)を参照）で作製した画像の位置合わせを、すでに位置合わせが終わっている画像をもとに行う場合はこの関数を使用する。

- SPM（Reorient Imageに相当）では変換行列ファイル（_sn.mat）が必要になるが、この関数はその変換行列を位置合わせ済みの画像のヘッダーから抽出して利用する。

- 3D Slicer等で位置合わせしたあとに、その画像をもとに他のdynamicとstaticの画像の位置合わせを行うことができる。

In [ ]:
def AffineTransformByAlignedImg(inputDir, inputFile, alignedImgFile, outputDir, prefix, suffix):
    #inputDir : 入力ファイルのあるディレクトリ
    #inputFile : 入力ファイル（4D NIfTI形式）
    #alignedImgFile : テンプレート画像に位置合わせが終わっている画像ファイル（パスおよび拡張子含めて指定）
    #outputDir : 出力ディレクトリの指定
    #prefix : 出力ファイル名の接頭語
    #suffix : 出力ファイル名の接尾語
    
    # import 4D NIfTI
    '''位置合わせしたい画像の読み込み'''
    niimg = load_img(inputDir + '/' + inputFile + '.nii')
    '''位置合わせ済み画像の読み込み'''
    alignedImgnii = load_img(alignedImgFile)
    
    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    ImgArryData = niimg.get_fdata()

    # 位置合わせ済み画像からaffine変換行列を抽出
    newAffineMatrix = alignedImgnii.header.get_best_affine()

    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換
    ConvertImg = nib.Nifti1Image(ImgArryData, affine=newAffineMatrix, header=new_header)
    print('Completed: ' + '/' + inputFile + '.nii')
    
    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + inputFile + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

使いどころとしては、まず上記のcreateMeanFramesImg（）関数で位置合わせ用の画像を作製し、3D Slicer等でテンプレート画像やCT画像と位置合わせを行います。

その後、3D Slicerで位置合わせの終わった画像を使ってDynamicおよびStatic画像の位置合わせを行います。

最初に関数に渡す引数（inputDir, inputFile, alignedImgFile, outputDir, prefix, suffix）を指定します。

ここでは前述の`createMeanFramesImg(inputDir, 'multiframe_test_data', slice(7, 30), outputDir, prefix, suffix)`で作った`m_multiframe_test_data.nii`というファイルが3D Slicerでテンプレート画像に対し位置合わせが終了したという前提で説明します。

この`m_multiframe_test_data.nii`を使って`test_data-SUV.nii`というファイルの位置合わせを行う場合は以下のようになります。

※　<u>引数`alignedImgFile`は絶対パスで指定してください。</u>

In [ ]:
inputDir = './dynamic'
outputDir = './dynamic'
prefix = 'r'
suffix = ''

AffineTransformByAlignedImg(inputDir, 'test_data-SUV', './dynamicm_multiframe_test_data.nii', outputDir, prefix, suffix)

# 05. SUV画像を%ID/ml画像にする変換する関数
---
あまり使うことはないと思いますが、参考までに載せておきます。

In [ ]:
def create_SUVimg2PrecentID_per_ml_img(inputDir, FileName, Weight_g, ID_MBq, outputDir, prefix, suffix):
    # inputDir: 
    # FileName: 拡張子は含めない
    # Weight_g: 
    # ID_MBq: Final Injection Dose (MBq) of tracer
    # outputDir: 出力ディレクトリパス
    # prefix: 出力ファイル名の接頭語
    # suffix: 出力ファイル名の接尾語
    
    # load a SUV NIfTI image
    tmpPathName = inputDir + '/' + FileName + '.nii'
    niimg = nib.load(tmpPathName)

    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    LoadImgArryData = niimg.get_fdata()

    # SUV画像をもとのBq/mlの画像に戻す
    Bq_pet_ml_Img = LoadImgArryData*((ID_MBq*1000)/(Weight_g/1000))

    # 最終投与量をもとに %ID/ml を計算する
    # もとの画素値がBq単位なので、最終投与濃度をMBq > Bqに変換して計算
    CalcImg = (Bq_pet_ml_Img/(ID_MBq*10**6))*100
    
    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換する
    ConvertImg = nib.Nifti1Image(CalcImg, None, header=new_header)
    print('Completed: ' + '/' + FileName + '.nii')
    
    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + FileName + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

最初に関数に渡す引数（inputDir, FileName, Weight_g, ID_MBq, outputDir, prefix, suffix）を指定します。

In [ ]:
in_outDir = './dynamic'
prefix = ''
suffix = '2PIDpml'

create_SUVimg2PrecentID_per_ml_img(in_outDir, 'r_test_data-SUV', 22.6, 5.44, in_outDir, prefix, suffix)

# 06. SUVr（リファレンスVOIで標準化した）画像を作製するための関数
---
SUV画像の値をリファレンスとなる値で割った画像を作製することが出来ます（いわゆるSUVr画像）。

SUVrの計算方法はいくつかあるようですが、ここでは2種類の関数を用意しています。

### 06-1. 一般的なSUVr
---
　PMOD等でも採用されている最も一般的な計算方法です。

　計算式は以下になります。

$$
= \frac{SUV\;value}{reference\;SUV\;value}
$$


In [ ]:
def createSUVrImg_PMOD(inputDir, FileName, refVOIval, outputDir, prefix, suffix):
    # inputDir: 
    # FileName: SUV画像を指定する。拡張子は含めない
    # refVOIval: 入力SUV画像のリファレンスにしたいVOIの値
    # outputDir: 出力ディレクトリパス
    # prefix: 出力ファイル名の接頭語
    # suffix: 出力ファイル名の接尾語
    
    # load a non-SUV NIfTI image
    tmpPathName = inputDir + '/' + FileName + '.nii'
    niimg = nib.load(tmpPathName)

    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    LoadImgArryData = niimg.get_fdata()

    # SUVrを計算する（PMODの計算式）
    CalcImg = LoadImgArryData/refVOIval
    
    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換する
    ConvertImg = nib.Nifti1Image(CalcImg, None, header=new_header)
    print('Completed: ' + '/' + FileName + '.nii')
    
    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + FileName + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

### 06-2. SAMITで採用されているSUVr
---
　SPMの動物PET用ツールボックスである[Small Animal Molecular Imaging Toolbox (SAMIT)](https://mic-umcg.github.io/samit/)で採用されている計算方法です。

　※　正確にはSAMITでは全脳（Whole brain）をリファレンスにしているため、SUVwと表現されます。<br>
　※　全脳をリファレンスにしなければならないという訳ではないと思われます。<br>
　※　ここではSAMITのマニュアルに沿って全脳で式を表したいと思います。<br>

　計算式は以下になります。<br>
<br>
$$
= \frac{SUV\;value}{Whole\;brain\;uptake\;value}
$$
<br>
　分子の`whole brain uptake value`はSUV変換前の値をさすため、以下のように表現することもできます。<br>
<br>
$$
=\frac{\frac{uptake\;in\;the\;Image}{\frac{Injected\;dose}{body\;weight}}}{Whole\;brain\;uptake\;value}
$$
<br>
$$
=\frac{uptake\;in\;the\;Image}{Whole\;brain\;uptake\;value}\times\frac{body\;weight}{Injected\;dose}
$$

In [ ]:
def createSUVrImg_SAMIT(inputDir, FileName, refVOIval, Weight_g, ID_MBq, outputDir, prefix, suffix):
    # inputDir: 
    # FileName: SUV画像を指定する。拡張子は含めない
    # refVOIval: 入力SUV画像のリファレンスにしたいVOIの値
    # Weight_g: 
    # ID_MBq: Final Injection Dose (MBq) of tracer
    # outputDir: 出力ディレクトリパス
    # prefix: 出力ファイル名の接頭語
    # suffix: 出力ファイル名の接尾語
    
    # load a non-SUV NIfTI image
    tmpPathName = inputDir + '/' + FileName + '.nii'
    niimg = nib.load(tmpPathName)

    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    LoadImgArryData = niimg.get_fdata()

    # 体重と最終投与量をもとにSUVrを補正計算する（SAMIT toolboxの計算式）
    # MBq -> kBq
    # g -> kg
    CalcImg = (LoadImgArryData/refVOIval)/((ID_MBq*1000)/(Weight_g/1000))
    
    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換する
    ConvertImg = nib.Nifti1Image(CalcImg, None, header=new_header)
    print('Completed: ' + '/' + FileName + '.nii')
    
    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + FileName + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

以下、実行例です。

どちらの関数も同じように使用することが出来ます。

In [ ]:
in_outDir = './static'
prefix = ''
suffix = 'r'

createSUVrImg_PMOD(in_outDir, 'test_data-SUV', 0.6087618, in_outDir, prefix, suffix)

# 07. 群の平均画像を作製するための関数
---
　群間比較を行った際、各群の平均画像が欲しい場合が多いと思われます。

　この関数は指定した個体データの平均画像を作製することが出来ます。

In [ ]:
def createMeanImg(inputDir, inputFileArray, outputDir, outputFile):
    #inputDir : 入力ファイルのあるディレクトリ
    #inputFileArray : 入力ファイル名（4D NIfTI形式）をnumpy.Array形式で与える
    #outputDir : 出力ディレクトリの指定
    #outputFile : 出力ファイル名
    
    # num of input file
    numInputFlile = inputFileArray.shape[0]
    
    # first import of 4D NIfTI
    niimg = load_img(inputDir + '/' + inputFileArray[0] + '.nii')
    print('Processed: ' + inputFileArray[0] + '.nii')
    
    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    ImgArryData = niimg.get_fdata()
    
    # next import of 4D NIfTI (loop)
    for inputFile in range(1, numInputFlile):
        nextNiimg = load_img(inputDir + '/' + inputFileArray[inputFile] + '.nii')        
        nextImgArryData = nextNiimg.get_fdata()

        # 加算
        ImgArryData = ImgArryData + nextImgArryData
        print('Processed: ' + inputFileArray[inputFile] + '.nii')
    
    # 平均
    MeanImg = ImgArryData/numInputFlile
    print('Total ' + str(numInputFlile) + ' images')
        
    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換
    ConvertImg = nib.Nifti1Image(MeanImg, affine=niimg.header.get_best_affine(), header=new_header)

    # seve mean image
    # .niiとして出力
    outputPathName = outputDir + '/' + outputFile + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))


以下、実行例です。

In [ ]:
in_outDir = './static'

createMeanImg(in_outDir, np.array(['test_data00-SUVr',
                                   'test_data01-SUVr',
                                   'test_data02-SUVr',
                                   'test_data03-SUVr']), in_outDir, 'Mean_test_data-SUVr')

# 08. 反転をするための関数(アフィン行列を修正して見た目だけ反転させる方法　→　元画像は反転しない)
---
　ClairvivoPET/CTでは必要ありませんが、機種によっては必要になる場合があるかと思います。

　反転をするための関数はこの08．と次項の09．の2種類を用意しました。

　この項ではヘッダーのアフィン行列を修正して見た目だけ反転させ、元画像は反転させない方法になります。

In [ ]:
def InvertAxis(inputDir, inputFile, TransformMatrix, outputDir, prefix, suffix):
    #inputDir : 入力ファイルのあるディレクトリ
    #inputFile : 入力ファイル（4D NIfTI形式）
    #TransformMatrix : affine変換行列（4x4）
    #outputDir : 出力ディレクトリの指定
    #prefix : 出力ファイル名の接頭語
    #suffix : 出力ファイル名の接尾語
    
    # import 4D NIfTI
    niimg = load_img(inputDir + '/' + inputFile + '.nii')
    
    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    ImgArryData = niimg.get_fdata()

    # affine変換後の回転成分を計算
    # 変換行列にオリジナルのaffine行列をかける、順番に注意
    newAffineMatrix = np.dot(niimg.header.get_best_affine(), TransformMatrix)

    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換
    ConvertImg = nib.Nifti1Image(ImgArryData, affine=newAffineMatrix, header=new_header)
    print('Completed: ' + '/' + inputFile + '.nii')

    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + inputFile + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

以下、実行例です。

ここではx軸で反転（左右反転）させる方法についての例です。

反転したい軸に合わせて引数`TransformMatrix`の行列を変更します（以下の`FlipLR`の部分）。

In [ ]:
# x軸で反転させる場合
FlipLR = np.array([[-1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])

in_outDir = './static'
prefix = ''
suffix = '_FlipLR'

InvertAxis(in_outDir, 'Mean_test_data-SUVr', FlipLR, in_outDir, prefix, suffix)

# 09. 反転をするための関数(アフィン行列は編集せずに元画像を反転させる方法)
---
　上記08．と似ていますが、こちらはヘッダーにあるアフィン行列情報は編集せずに元画像を反転させる方法となります。

In [ ]:
def InvertAxisV2(inputDir, inputFile, TransformAxis, outputDir, prefix, suffix):
    #inputDir : 入力ファイルのあるディレクトリ
    #inputFile : 入力ファイル（4D NIfTI形式）
    #TransformAxis : x = 0, y = 1, z = 2
    #outputDir : 出力ディレクトリの指定
    #prefix : 出力ファイル名の接頭語
    #suffix : 出力ファイル名の接尾語
    
    # import 4D NIfTI
    niimg = load_img(inputDir + '/' + inputFile + '.nii')
    
    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    ImgArryData = niimg.get_fdata()
    
    # 反転処理
    ImgArryData = np.flip(ImgArryData, TransformAxis)

    # affine変換後の回転成分を計算
    # 変換行列にオリジナルのaffine行列をかける、順番に注意
    newAffineMatrix = niimg.header.get_best_affine()

    # 読み込みデータからヘッダーのコピーを作製
    new_header = niimg.header.copy()

    # NumpyArrayからNIfTI形式に変換
    ConvertImg = nib.Nifti1Image(ImgArryData, affine=newAffineMatrix, header=new_header)
    print('Completed: ' + '/' + inputFile + '.nii')

    # .niiとして出力
    outputPathName = outputDir + '/' + prefix + inputFile + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

以下、実行例です。

08．と同様にx軸で反転させる場合を示します。

※　上記08．のパラメーター変数`FlipLR`が定義済みという前提です。

In [ ]:
in_outDir = './static'
prefix = ''
suffix = '_FlipLR'

# x軸で反転させる場合
InvertAxisV2(in_outDir, 'Mean_K2_Stc_PreScan_n4', 0, in_outDir, prefix, suffix)

# 10. 差分画像を作製するための関数
---
　2つの画像を読み込み引き算することで差分画像を作製します。

　2つの画像は同じディレクトリに置いておく必要があります。

In [ ]:
def CalcDifferenceImg(inputDir, InputFileNameA, InputFileNameB, outputDir, outputFileName):
    # inputDir : 入力画像のパス
    # InputFileNameA : 入力画像のファイル名（引かれる側）
    # InputFileNameB : 入力画像のファイル名（引く側）
    #      →　A - B　となる
    # outputDir : 差分画像の出力先パス
    # outputFileName : 差分画像のファイル名
    
    # import 4D NIfTI
    tmpLoadImgA = nib.load(inputDir + '/' + InputFileNameA + '.nii')
    tmpLoadImgB = nib.load(inputDir + '/' + InputFileNameB + '.nii')
    
    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    LoadImgArryDataA = tmpLoadImgA.get_fdata()
    LoadImgArryDataB = tmpLoadImgB.get_fdata()
    
    # 2つの画像の差分を計算する
    CalcImg = LoadImgArryDataA - LoadImgArryDataB
    
    # 読み込んだ画像からアフィン変換に必要な行列を取得
    # NumpyArrayからNIfTI形式に変換する
    ConvertImg = nib.Nifti1Image(CalcImg, affine=tmpLoadImgA.header.get_best_affine())
    
    # .niiとして出力
    outputPathName = outputDir + '/' + outputFileName + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

以下、実行例です。

ここでは `test_data00-SUVr` から `test_data01-SUVr` を引いた画像を作る場合を示します。


In [ ]:
'''input Parameters'''
inputDir = './static'
InputFileName1 = 'test_data00-SUVr'
InputFileName2 = 'test_data01-SUVr'

'''output Parameters & Execute'''
outputDir = './static'
outputFileName = 'DifferenceImg_test_data00-01_SUVr'

CalcDifferenceImg(inputDir, InputFileName1, InputFileName2, outputDir, outputFileName)

# 11. 割合（百分率）画像を作製するための関数
---
　2つの画像を読み込み割り算することで割合（百分率）画像を作製します。

　2つの画像は同じディレクトリに置いておく必要があります。

In [ ]:
def CalcRatioImg(inputDir, InputFileNameA, InputFileNameB, outputDir, outputFileName):
    # inputDir : 入力画像のパス
    # InputFileNameA : 入力画像のファイル名（分子となる画像）
    # InputFileNameB : 入力画像のファイル名（分母となる画像）
    #      →　A / B　となる
    # outputDir : 割合画像の出力先パス
    # outputFileName : 割合画像のファイル名

    # import 4D NIfTI
    tmpLoadImgA = nib.load(inputDir + '/' + InputFileNameA + '.nii')
    tmpLoadImgB = nib.load(inputDir + '/' + InputFileNameB + '.nii')
    
    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    LoadImgArryDataA = tmpLoadImgA.get_fdata()
    LoadImgArryDataB = tmpLoadImgB.get_fdata()
    
    # 画像Bに対する画像Aの割合を計算する
    CalcImg = (LoadImgArryDataA/LoadImgArryDataB)*100
    
    # 読み込んだ画像からアフィン変換に必要な行列を取得
    # NumpyArrayからNIfTI形式に変換する
    ConvertImg = nib.Nifti1Image(CalcImg, affine=tmpLoadImgA.header.get_best_affine())
    
    # .niiとして出力
    outputPathName = outputDir + '/' + outputFileName + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

以下、実行例です。

ここでは `test_data00-SUVr` を `test_data01-SUVr` で割った画像を作る場合を示します。


In [ ]:
'''input Parameters'''
inputDir = './static'
InputFileName1 = 'test_data00-SUVr'
InputFileName2 = 'test_data01-SUVr'

'''output Parameters & Execute'''
outputDir = './static'
outputFileName = 'RatioImg_test_data00per01_SUVr'

CalcRatioImg(inputDir, InputFileName1, InputFileName2, outputDir, outputFileName)

# 12. マスク適用画像を作製するための関数
---
　マスク画像とマスクをかけたい（適用したい）画像を読み込みます。

　2つの画像は同じディレクトリに置いておく必要があります。

　マスク画像とマスクを適用したい画像のshapeが一致していないと処理を実行することができません。

　shapeが一致しない場合はClopping処理を行い一致させるようにしてください。

In [ ]:
def CalcMaskImg(MaskFilePathName, inputDir, inputFile, outputDir, prefix, suffix):
    # MaskFilePathName : マスク画像のパス名含むファイル名（フルパス）
    # inputDir : マスク適用画像のあるディレクトリ
    # inputFile : マスク適用画像のファイル名
    # outputDir : マスク適用画像の出力ディレクトリ
    #　prefix : 出力ファイル名の接頭語
    #　suffix : 出力ファイル名の接尾語

    tmpLoadImgMask = nib.load(MaskFilePathName)
    tmpLoadImg = nib.load(inputDir + '/' + inputFile + '.nii')
    
    # 読み込んだ画像はNiBabel形式なのでNumpy.Array形式で扱えるようにする
    LoadImgArryDataMask = tmpLoadImgMask.get_fdata()
    LoadImgArryData = tmpLoadImg.get_fdata()
    
    # マスク画像とマスクを適用したい画像のshapeが一致していないと処理ができない
    # 一致していない場合は以下の1行を編集して使用すること
    # np.delete()の第2引数は削除したいスライス位置、第3引数は0=行、1=列、2=行列の削除を表す
    # 通常はこの1行はコメントアウトしておくこと
    LoadImgArryDataMask = np.delete(LoadImgArryDataMask, 96, 2) # 今回はマスク以外のテンプレートが(96,120,96)なのでそれに合わせた
    
    # 上記のイレギュラー処理が必要な場合はコンソールに警告を出す
    if (tmpLoadImg.shape == tmpLoadImgMask.shape):
        pass
    else:
        print('******************************************************')
        print('：：：警告：：：')
        print('')
        print('マスク画像とマスクを適用したい画像のshapeが一致していません！')
        print('関数内のnp.delete()を適切に編集してshapeが同じになるようにしてください。')
        print('******************************************************')
        print('')

    # 画像Bに対する画像Aの割合を計算する（マスクを欠けた部分を0にする）
    CalcImg = LoadImgArryData*LoadImgArryDataMask
    
    # 読み込みデータからヘッダーのコピーを作製
    new_header = tmpLoadImg.header.copy()

    # 読み込んだ画像からアフィン変換に必要な行列を取得
    # NumpyArrayからNIfTI形式に変換する
    ConvertImg = nib.Nifti1Image(CalcImg, affine=tmpLoadImg.header.get_best_affine(), header=new_header)
    
    print('Completed: ' + '/' + inputFile + '.nii')

    # .niiとして出力
    outputDir + '/' + prefix + inputFile + suffix + '.nii'
    outputPathName = outputDir + '/' + prefix + inputFile + suffix + '.nii'
    nib.save(ConvertImg, os.path.join(outputPathName))

以下、実行例です。

ここではSAMITのラット解析用に用意されているマスク画像 `Schwarz_intracranialMask.nii` を利用した場合を示します。

In [ ]:
MaskFilePathName = './Schwarz_intracranialMask.nii' # マスク画像のパス

in_outDir = './static'
prefix = 'Msk_'
suffix = ''

CalcMaskImg(MaskFilePathName, in_outDir, 'test_data00-SUVr', in_outDir, prefix, suffix)